# Házidolgozat: UFC adatelemzés

Házidolgozatom során a világ legnagyobb ketrecharc szervezetének, az UFC atlétáinak adataival dolgoztam. A hivatalos weboldaljukról szereztem be az adatokat. Mivel eleinte a weboldal csak kb. 10 sportolót mutat, a load more gombra kell kattintani, hogy még tizet megmutasson. Mivel összesen 2857 atléta van az UFC alkalmazásában írtam egy scriptet a Selenium package segítségével, ami megnyitja a cromeot és folyamatosan rákattint a load more gombra és így ki tudja szedni az összes adatot a weboldalról. Többször is megszakadt a data scrapelésem az adatbányászat különböző pontjain, így rájöttem, hogy a site dob le agressive data scraping miatt. Emiatt a random package segítségével véletlenszerű időintervallumot generáltam a klikkelésekre, hogy emberszerűbb legyen a scrape és ne hajítson le a weboldal. Elsőre leszedte 1-3 mp közötti időintervallumos kattintásokkal az egész dataframe-t, kb. 15 perc volt. Másodjára ugyanezen beállításokkal kb. 600 lescrapelt küzdősportoló után ledobott a site megint, úgyhogy 2-5 mp-re álltítottam be, így jelenlegi beálíltások mellett 25 perc alatt szedi le az adatokat. Nyilván utólag megállapítva ez nem a legoptimálisabb használandó adathalmaz volt, de elég mélyen belemerűltem a témába és jó irányba indultam el az elején ezzel a megoldással, így sikerült valami újat tanulnom. A scrapelt datát rögtön lementettem egy excelbe, ami egyfajta ,,biztonsági mentési" pontként szolgált nekem, hogy ne kelljen állandóan újból 20 percet várnom az adatokra, ha elrontok valamit. Ezt a fájlt is mellékelem a notebook mellé, amennyiben nincsen kedve sokat várnia az adatokra a tanárnőnek. Az adathalmazunkról annyit lehet tudni, hogy minden harcosnak 4 változója van az adatokban. A neve, súlycsoportja, beceneve és mérlege. A mérleg oszlopot szétszedtem 3 különböző oszlopra (győzelem,vereség,döntetlen) és ezzel végeztem többnyire kisebb adatvizualizációkat és elemzéseket. Azon nem kell meglepődni, hogy jelentős túlsúlyban vannak a győzelmek a vereségekhez képest, ennél a szervezetnél a világ legjobbjai versenyeznek, és komoly mérleget harcolnak össze maguknak kisebb szervezeteknél, mielőtt felkeltenék az UFC figyelmét és leigazolnák őket ide. Döntetlenek nagyon ritkán fordulnak elő a sport pontozási mivoltja miatt. Három menetes meccseket vívnak átlagos párosításokon, címmeccseken ötöt és a bírók a meneteket többnyire 10-9 arányban pontozzák. Az adatok egyaránt tartalmaznak férfi és női versenyzőket, utóbbiaknak az adatokban a súlycsoportnál ez fel van tüntetve. Azt kérem vegye figyelembe a tanárnő, hogy én a GTK-ra járok alapból és nem rendelkezem olyan mértékű statisztikai tudással, mint a többiek.  

Winkler Dávid

Első lépésként előre betöltjük az összes szükséges packaget, amit később használni fogunk.

In [ ]:
import ssl # Importáljuk az összes szükséges packaget, mindent feltelepítettem a saját gépemre, a seleniumot is.
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib
import time  
import random # Ez a randomszám generáláshoz kell, hogy ne dobjon le a weboldal agressive data scraping miatt
from selenium import webdriver # Selenium packaget használjuk a script írásához, ami folyamatosan rákattint a load more gombra 
from selenium.webdriver.common.by import By # Többféle változata is volt a kódnak, nem biztos hogy ezek közül kell még mindegyik
from selenium.webdriver.support.ui import WebDriverWait # De biztosra menve bennehagyom mindegyiket a kódban
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys



Ezután a selenium package segítségével megszerezzük az összes ketrecharcosnak az adatait.

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.ufc.com/athletes/all") # Ez a weboldal címe, ahonnan leszedtem az adatokat.

# Megtalálja a "Load More" gombot és addig klikkel rá, amíg be nem tölti a weboldal az összes atlétát
while True:
    try:
        load_more_button = driver.find_element(By.CLASS_NAME, "pager__item") # Ez a load more gomb
        load_more_button.click()
        delay = random.uniform(2, 5)  # Véletlenszerű szünetet generál kettő klikkelés között  2 és 5 másodperc hosszan
        time.sleep(delay) # 1-3 mp között, volt, hogy leszedte az egészet, de olyan is, hogy ledobott, ez így kb.20 perc
    except (NoSuchElementException, StaleElementReferenceException):
        break

# Begyűjtjük az atléták adatait
soup = BeautifulSoup(driver.page_source, "html.parser")
elemek = soup.find_all('li', {'class': 'l-flex__item'})
data = []
for elem in range(len(elemek)):
    kislista = []

    név = elemek[elem].find('span', {'class': 'c-listing-athlete__name'}) # A kódnak ez a része begyűjti az atléták neveit
    név2 = név.text.strip() if név else None
    kislista.append(név2)

    súlycsoport = elemek[elem].find('span', {'class': 'c-listing-athlete__title'}) # Ez a része a súlycsoportját
    súlycsoport2 = súlycsoport.text.strip() if súlycsoport else None
    kislista.append(súlycsoport2)

    try:
        becenév = elemek[elem].find('div', {'class': 'field field--name-nickname field--type-string field--label-hidden'})
        becenév2 = becenév.text.strip() if becenév else None # Ez a beceneveiket, nem rendelkezik mindenki ilyen adattal
        kislista.append(becenév2)
    except:
        kislista.append(None)

    mérleg = elemek[elem].find('span', {'class': 'c-listing-athlete__record'}) # Ez pedig a mérlegüket (győzelem,vereség,döntetlen)
    mérleg2 = mérleg.text.strip() if mérleg else None
    kislista.append(mérleg2)

    data.append(kislista)

# Megcsináljuk a DataFrame-t
df = pd.DataFrame(data, columns=["Név", "Súlycsoport", "Becenév", "Mérleg"])

# Megnézzük a DataFrame-t
df.head(2857)

Excelként letöltjük a dataframe-t.

In [ ]:
df.to_excel('ufc.xlsx', index=False) # Excelként letöltjük a dataframe-t, ezt el is küldöm majd

In [ ]:
df = pd.read_excel('ufc.xlsx') # Itt tudjuk betölteni, egy elrontott kód után én is itt töltöttem vissza
df.head(10)

Kisebb időközönként becsúszott egy-egy üres sor az adatokba, mert ,,üres" helyek is vannak a weboldalon. Ezeket el kell távolítani.

In [ ]:
# Megtisztítjuk az adathalmazunkat, azoktól a soroktól melyeknél minden érték üres
df.dropna(how='all', inplace=True)

# Megnézzük a javított DataFrame-t
df.head(3200) # Amikor ezt készítem 2857 fighternek kell lennie, ha többet írok a headbe tudom ellenőrizni ezt a számot

Mivel a súlycsoportok angolul vannak, szeretnénk ezt lefordítani. Más ketrecharc egyesületeknél léteznek szalmasúlyú és szupernehéz férfiak, valamint nehezebb női súlycsoportok is. Bár a súlycsoportok más organizációknál is ugyanezen határral léteznek, mindig az adott szervezet dönti el, hogy igazol-e ezekbe versenyzőket egymás elleni versenyezteztetés céljából.  

In [ ]:
# Meghatározzuk a súlycsoportok magyar neveit
súlycsoport_lefordítás = {
    'Flyweight': 'Légsúly', # 52,2-56,7 kg
    'Bantamweight': 'Harmatsúly', # 56,7-61,2 kg
    'Featherweight': 'Pehelysúly', # 65,8-70,3 kg
    'Lightweight': 'Könnyűsúly', # 70,3-74,8 kg
    'Welterweight': 'Váltósúly', # 74,8-77,1 kg
    'Middleweight': 'Középsúly', # 77,1-83,9 kg
    'Light Heavyweight': 'Cirkálósúly', # 83,9-93,0 kg
    'Heavyweight': 'Nehézsúly', # 93,0-120,2 kg
    'Women\'s Strawweight': 'Női Szalmasúly', # < 52,2 kg
    'Women\'s Flyweight': 'Női Légsúly', # 52,2-56,7 kg
    'Women\'s Bantamweight': 'Női Harmatsúly', # 56,7-61,2 kg
    'Women\'s Featherweight': 'Női Pehelysúly', # 65,8-70,3 kg
    'Women\'s Bantamweight\nWomen\'s Featherweight': 'Női Harmatsúly és Pehelysúly' # Egy későbbi kód során lett felfedezve
}

# Helyettesítjük a súlycsoportok értékeit a magyar neveikkel
df['Súlycsoport'] = df['Súlycsoport'].replace(súlycsoport_lefordítás)

In [ ]:
df.head() # Megnézzük sikerrel jártunk-e

A Mérleg oszlopot felosztjuk 3 oszlopra, hogy tudjunk elemzéseket végezni.

In [ ]:
# Felosztjuk a 'Mérleg' oszlopot három különálló oszlopra, hogy tudjunk elemzéseket végezni az adathalmazunkon
split_data = df['Mérleg'].str.split('-', expand=True)
df['Győzelem'] = split_data[0]
df['Vereség'] = split_data[1]
df['Döntetlen'] = split_data[2].str.rstrip(' (W') # A W-t ki kell szedni a döntetlen oszlopból

# Kiszedjük a 'Mérleg' oszlopot
df.drop('Mérleg', axis=1, inplace=True)

# Megnézzük a javított DataFrame-t
df.head()

Szeretnénk azt megtudni, milyen győzelmi aránnyal rendelkeznek együttesen az UFC harcosai. Ezt vizualizáljuk egy kördiagrammon.

In [ ]:
import matplotlib.pyplot as plt

# Kiszámoljuk a harcosok összesített győzelmeit, vereségeit és döntlenjeiket
összes_győzelem = df['Győzelem'].astype(int).sum()
összes_vereség = df['Vereség'].astype(int).sum()
összes_döntetlen = df['Döntetlen'].astype(int).sum()

# Csinálunk egy listát a különböző kimenetelek érétkeinek
kimenetelek = [összes_győzelem, összes_vereség, összes_döntetlen]

# Csinálunk egy listát a különböző eredmények cimkézéseihez
címkék = ['Győzelmek', 'Vereségek', 'Döntetlenek']

# Csinálunk egy kördiagrammot
plt.pie(kimenetelek, labels=címkék, autopct='%1.1f%%', startangle=90)

# Adunk neki egy címet
plt.title('Összesített győzelmek, vereségek, és döntetlenek százalékos aránya a harcosok mérlegeiben')

# Megjelenítjük az ábrát
plt.show()

Ezután a UFC harcosainak nem az összesített győzelmi arányára, hanem a kombinált mérlegére vagyunk kiváncsiak. Ezt egy oszlopdiagrammon szeretnénk vizualizálni.

In [ ]:
import matplotlib.pyplot as plt

# Kiszámoljuk a kombinált mérlegét az atlétáknak
kombinált_eredmények = df[['Győzelem', 'Vereség', 'Döntetlen']].astype(int).sum()

# Csinálunk egy oszlopdiagrammot a kombinált eredményeknek
plt.figure()
plt.bar(kombinált_eredmények.index, kombinált_eredmények.values)
plt.ylabel('Gyakoriság')
plt.title('Összesített mérleg')

# Megjelenítjük az ábrát
plt.show()

# Kiszámoljuk a kombinált mérlegét az atlétáknak
kombinált_mérleg = df[['Győzelem', 'Vereség', 'Döntetlen']].astype(int).sum()
kombinált_mérleg_str = f"{kombinált_mérleg['Győzelem']}-{kombinált_mérleg['Vereség']}-{kombinált_mérleg['Döntetlen']}"

# Kiprinteljük a harcosok összesített mérlegét
print("Összesített mérleg:", kombinált_mérleg_str)

Szeretnénk megtudni mely súlycsoportban hányan versenyeznek az UFC keretein belül.

In [ ]:
df['Súlycsoport'].value_counts() # Megnézzük mely súlycsoportokban hány harcost foglalkoztat a UFC

Vizualizáljuk ezt oszlopdiagrammon is.

In [ ]:
df['Súlycsoport'].value_counts().plot(kind='bar') # Vizualizáljuk oszlopdiagrammon az előző eredményeket

Mivel nem jön ki a matek valószínűsíthetően egyes ketrecharcosoknak nincsen megadva súlycsoport az adatokban. Szeretnénk megtudni hány ilyen versenyző van. (2786 jön ki, ha az összes számot összeadjuk és 2857 atlétának kell lennie elvileg)

In [ ]:
# Count the number of people without weight classes
súlycsoport_nélküli = df['Súlycsoport'].isna().sum()

# Kiprinteljük az eredményt
print("Megjelölt súlycsoport nélküli atléták az adatokban:", súlycsoport_nélküli)

Megkeressük a hölgyet, aki kettő súlycsoportban is versenyez egyszerre.

In [ ]:
df_kettős = df[df['Súlycsoport'].str.contains("Női Harmatsúly") & df['Súlycsoport'].str.contains("Pehelysúly")]
dupla_súlycsoportos = df_kettős.iloc[0] # Megkeressük a hölgyet, aki kettő súlycsoportban versenyez egyszerre

dupla_súlycsoportos

Arra vagyunk kiváncsiak, hányan versenyeznek a UFC-ben a női és a férfi súlycsoportokban. Ezt szeretnénk oszlopdiagrammon vizualizálni. (Ebben a súlycsoport nélküli 71 természetesen nincsen benne)

In [ ]:
import matplotlib.pyplot as plt

# Megszámoljuk az összes súlycsoportnak az előfordulását
súly_csoportok_számolása = df['Súlycsoport'].value_counts()

# Elválasztjuk a nőket a férfiaktól
női_súly_csoportok = súly_csoportok_számolása[súly_csoportok_számolása.index.str.startswith('Női')]
férfi_súly_csoportok = súly_csoportok_számolása[~súly_csoportok_számolása.index.str.startswith('Női')]

# Csinálunk egy oszlopdiagrammot
plt.bar(['Női', 'Férfi'], [női_súly_csoportok.sum(), férfi_súly_csoportok.sum()], color=['pink', 'blue'])

# Beállítjuk a címet és a magyarázó szövegeket
plt.xlabel('Nem')
plt.ylabel('Harcosok száma')
plt.title('Harcosok száma nem szerint')

# Megjelenítjük az eredményeket
for i, count in enumerate([női_súly_csoportok.sum(), férfi_súly_csoportok.sum()]):
    plt.text(i, count, str(count), ha='center', va='bottom')

# Megjelenítjük a diagramot
plt.show()

A UFC-ben a harcosok nagy része rendelkezik becenévvel, amivel a szervezet promózza őket, és ők reklámozzák önmagukat. Kiváncsiak vagyunk arra, hogy hány harcos rendelkezik ilyen adattal és hányan nem. Ezt szintén szeretnénk vizualizálni.

In [ ]:
import matplotlib.pyplot as plt

# Rászűrűnk az adathalmazban azokra, akiknek van beceneve
van_beceneve = df.dropna(subset=['Becenév'])

# Rászűrűnk azokra az adathalmazban, akiknek nincsen beceneve
nincs_beceneve = df[df['Becenév'].isna()]

# Megszámoljuk a beceneves és nem beceneves harcosokaz
van_beceneve_szám = len(van_beceneve)
nincs_beceneve_szám = len(nincs_beceneve)

# Készítünk egy oszlopdiagrammot
plt.figure(figsize=(8, 6))  # Beállítjuk a méreteket preferencia szerint

# Plotoljuk a becenévvel rendelkező harcosokat és megjelenítjük az eredményeket
plt.bar('Becenévvel rendelkezik', van_beceneve_szám, color='blue')
plt.text('Becenévvel rendelkezik', van_beceneve_szám, str(van_beceneve_szám), ha='center', va='bottom')

# Plotoljuk a becenév nélküli harcosokat és megjelenítjük az eredményeket
plt.bar('Becenévvel nem rendelkezik', nincs_beceneve_szám, color='red')
plt.text('Becenévvel nem rendelkezik', nincs_beceneve_szám, str(nincs_beceneve_szám), ha='center', va='bottom')

# Beállítjuk a magyarázatokat és a címet
plt.xlabel('Becenév')
plt.ylabel('Mennyiség')
plt.title('Becenévvel rendelkező és nem rendelkező harcosok száma')

# Megjelenítjük a diagrammot
plt.show()

Szeretnénk megtalálni a leggyakoribb beceneveket, amit a harcosok felvettek.

In [ ]:
import matplotlib.pyplot as plt

# Csoportosítjuk a DataFramet Becenév szerint és megszámoljuk a gyakoriságot
becenév_számlálás = df.groupby('Becenév').size().reset_index(name='Gyakoriság')

# Rászűrünk, hogy csak a többszörös előfordulás jelenjen meg
gyakori_becenév = becenév_számlálás[becenév_számlálás['Gyakoriság'] > 2] # Ez azért van 3 vagy többre állítva, mert 1+-nál egybefolynak a dolgok

# Létrehozzuk a diagrammot
plt.figure(figsize=(10, 6))  # Preferencia alapján beállítjuk a méreteket

# Ábrázolja a harcosok számát ugyanazzal a becenévvel és jelenítse meg az értékeket
plt.bar(gyakori_becenév['Becenév'], gyakori_becenév['Gyakoriság'], color='blue')

# Beállítja a magyarázatokat és a címet
plt.xlabel('Becenév')
plt.ylabel('Gyakoriság')
plt.title('Harcosok ugyanolyan becenevei')

# 90 fokkal elfordítjuk x tengely magyarázatait jobb láthatóság miatt
plt.xticks(rotation=90)

# Megjelenítjük a diagrammot.
plt.show()

Az előző diagrammon láttuk, hogy néhány nevet elég sokan használnak, valamint az 5. leggyakrabban használt becenév esetén holtverseny van. Szeretnénk megjeleníteni ezért a 6 leggyakrabban használt becenevet felvevő harcosoknak az adatait, hogy láthassuk kik ezek a küzdősportolók.

In [ ]:
# Csoportosítsuk a DataFrame-et "Becenév" alapján, és számoljuk meg az előfordulások számát
becenev_elofordulasok = df['Becenév'].value_counts()

# Válasszuk ki a N leggyakoribb "Becenév" értéket
top_n = 6  # Itt tudjuk megadni, hány leggyakoribb értéket szeretnénk figyelembe venni
top_becenev = becenev_elofordulasok.head(top_n)

# Hozzunk létre egy tuple listát a "Becenév" értékkel és a hozzá tartozó harcosokkal
eredmeny = [(becenev, df[df['Becenév'] == becenev]) for becenev in top_becenev.index]

# Írjuk ki az adatokat
for becenev, harcosok in eredmeny:
    print(f"Becenév: {becenev}")
    print("Harcosok:")
    harcosok_megjelenites = harcosok[['Név', 'Súlycsoport', 'Győzelem', 'Vereség', 'Döntetlen']]
    print(harcosok_megjelenites.to_string(index=False))
    print()